In [98]:
from bs4 import BeautifulSoup
from requests import get
import re
import xlwt 
from xlwt import Workbook
import pandas as pd

In [99]:
# Splits the keyword based on space and inserts + between each word (to make it compatible to the url)
# Change the Keyword to match requirements
keyword = "machine learning"
search = keyword.split()
if len(search) > 1:
    search_key = ""
    for each in search:
        search_key += each + "+"
    search_key = search_key[0:len(search_key)-1]
else:
    search_key = search[0]


In [100]:
url = "https://www.meetup.com/find/events/?allMeetups=false&keywords="+ search_key + "&radius=10&userFreeform=san+fra&mcId=c94101&gcResults=San+Francisco%2C+CA%2C+USA%3AUS%3ACalifornia%3ASan+Francisco+County%3ASan+Francisco%3Anull%3Anull%3A37.7749295%3A-122.4194155&change=yes&eventFilter=mysugg"
#url = "https://www.meetup.com/find/events/?allMeetups=false&keywords=python&radius=10&userFreeform=San+Francisco%2C+California%2C+USA&mcId=c94101&mcName=San+Francisco%2C+CA&eventFilter=all"
response = get(url)

In [101]:
data = BeautifulSoup(response.text, "html.parser")

In [102]:
data1 = data.findAll("a", itemprop="url")

In [103]:
# Removes the duplicate url as we get each url two times when we get all the <a> tags
data1 = data.findAll(["a"], itemprop="url")
weblinks = []
i = 1
for link in data1:
    if i%2 == 0:
        weblinks.append(link.get("href"))
    i += 1

In [104]:
eventname = data.findAll("span", itemprop="name")

In [105]:
# Separates the event name and the host name
group = []
event = []
i = 1
for e in eventname:
    if i%2 == 1:
        group.append(e.get_text())
    else:
        event.append(e.get_text())
    i += 1

In [106]:
name = data.findAll("time", itemprop="startDate")

In [107]:
# regular expression to extract date from the date time format
dateregex = re.compile("\d{4}-\d{2}-\d{2}")

In [108]:
# extracts date from the datetime tags
eventdate = []
eventtime = []
for i in name:
    if(i.get("datetime") != None):
        eventdate.append(dateregex.findall(str(i.get("datetime")))[0])
    eventtime.append(i.get_text().strip('\n').replace('\n', ' '))

In [109]:
df = pd.DataFrame(list(zip(event, group, eventdate, eventtime, weblinks)), 
                  columns = ["Event", "Group Name", "Date", "Time", "Weblink"])

In [110]:
# Saves the data to a csv file
export_csv = df.to_csv (r'events.csv', index = None, header=True)